About the Dataset:

id: unique id for a news article
title: the title of a news article
author: author of the news article
text: the text of the article; could be incomplete
label: a label that marks whether the news article is real or fake:
    1: Fake news
    0: real News

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asli_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

`stopwords.words('english')` fonksiyonu, İngilizce'de sık kullanılan ve anlam taşımayan kelimeleri (ör. "the", "is", "and") bir liste olarak döndürür. Bu kelimeler genellikle metin temizleme işlemlerinde çıkarılarak modelin gereksiz verilerle uğraşması önlenir, bu da daha hızlı ve doğru bir analiz sağlar. Özellikle doğal dil işleme (NLP) ve makine öğrenmesi projelerinde, metni sadeleştirip anlamlı bilgilere odaklanmak için kullanılır. Ancak, metnin bütünlüğü kritikse (ör. şiir veya konuşma analizi), bu kelimeleri korumak tercih edilebilir.

In [57]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [58]:
news_dataset = pd.read_csv('News_dataset.csv')

In [59]:
news_dataset.shape

(20800, 5)

In [60]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1



1. **`id`**: Her makale için benzersiz bir kimlik numarası (analiz için önemsiz).
2. **`title`**: Haber başlığı, konunun özetini verir ve analizde faydalıdır.
3. **`author`**: Makalenin yazarı, bazı durumlarda sahte haberleri ayırt etmek için kullanılabilir.
4. **`text`**: Haber metninin tam içeriği, modelin sınıflandırma yapmasında en önemli özniteliktir.
5. **`label`**: Haber türünü belirtir; `1` sahte haber, `0` gerçek haber.



In [61]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [62]:
# Eksik (NaN) değerleri boş bir string ('') ile dolduruyoruz
news_dataset = news_dataset.fillna('')

# Yeni bir sütun ('content') oluşturuyoruz. Bu sütun, 'author' ve 'title' sütunlarının birleştirilmiş halini içerir
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']

# Yeni oluşturulan 'content' sütununun tüm değerlerini ekrana yazdırıyoruz
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [63]:
# 'label' sütununu bağımsız değişkenlerden (X) çıkarıyoruz ve geriye kalan sütunlardan yeni bir veri seti oluşturuyoruz
X = news_dataset.drop(columns='label', axis=1)

# 'label' sütununu hedef değişken (Y) olarak ayırıyoruz
Y = news_dataset['label']

# Bağımsız değişkenleri (X) ekrana yazdırıyoruz
print(X)

# Hedef değişkeni (Y) ekrana yazdırıyoruz
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [64]:
port_stem = PorterStemmer()

**`PorterStemmer()`**:
   - Porter Stemming Algoritması'nı kullanarak kelimeleri köklerine indirger.
   - Örneğin, "running", "runner", ve "runs" kelimelerini aynı köke ("run") dönüştürür.

### Neden Kullanılır?
- **Doğal Dil İşleme (NLP)**: Metinlerdeki kelimeleri sadeleştirerek (örn. "running" -> "run") farklı biçimlerde yazılmış aynı anlamdaki kelimeleri birleştirir.
- **Boyut Azaltma**: Özellik uzayını küçülterek modelin daha verimli çalışmasını sağlar.
- **Bilgi Kaynağı Birleştirme**: Benzer anlam taşıyan kelimeleri aynı köke indirgeyerek modelin daha anlamlı tahminler yapmasına yardımcı olur.

Bu satır, kelimelere stemming uygulanacak bir adımın ön hazırlığını yapar.

In [65]:
def stemming(content):
    # 'content' metnindeki harf dışındaki tüm karakterleri (noktalama işaretleri, sayılar) boşluk ile değiştiriyoruz
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    
    # Metni küçük harflere dönüştürüyoruz, böylece aynı kelimeler büyük/küçük harf farkı olmadan eşit kabul edilir
    stemmed_content = stemmed_content.lower()
    
    # Metni kelimelere ayırıyoruz (boşluklara göre)
    stemmed_content = stemmed_content.split()
    
    # Burada, her kelimeyi port_stem.stem() fonksiyonu ile kök haline getiriyoruz. Aynı zamanda stopwords (durdurma kelimeleri) listesindeki kelimeler çıkarılıyor. Yalnızca anlamlı kelimeler ve kökler kalır.
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    
    # Kök haline getirilmiş kelimeleri tekrar birleştirerek bir metin oluşturuyoruz
    stemmed_content = ' '.join(stemmed_content)
    
    # İşlem tamamlandığında temizlenmiş metni geri döndürüyoruz
    return stemmed_content

# apply() fonksiyonu ile, stemming() fonksiyonu news_dataset['content'] sütunundaki her bir metne uygulanır.
news_dataset['content'] = news_dataset['content'].apply(stemming)

# 'content' sütunundaki son haliyle metinleri yazdırıyoruz
print(news_dataset['content'])

#Bu işlem, haber metinlerindeki gereksiz kelimeleri çıkarıp, kelimeleri köklerine indirger, böylece metinler sadeleştirilir ve analiz için daha uygun hale gelir.

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


Bu satırda yapılan işlem, kök haline getirilmiş kelimelerin tekrar birleştirilmesidir. 

### Açıklama:
```python
stemmed_content = ' '.join(stemmed_content)
```

- **`stemmed_content`**: Bu, kelimelerden oluşan bir listedir. Bu liste, köklerine indirgenmiş ve stopword (gereksiz kelimeler) çıkarılmış kelimelerden oluşur. Örneğin, `'runn', 'play', 'footbal'` gibi kökler içerebilir.
  
- **`' '.join(stemmed_content)`**: 
   - **`join()`**: Bu fonksiyon, listedeki elemanları (kelimeleri) birleştirir.
   - `' '` (boşluk karakteri): Bu, kelimelerin arasında bir boşluk bırakılmasını sağlar. Yani, kelimeleri aralarına boşluk koyarak birleştirir.
  
#### Örnek:
Eğer `stemmed_content` şu şekilde bir liste içeriyorsa:
```python
['run', 'fast', 'play', 'footbal']
```
`' '.join(stemmed_content)` işlemi şu şekilde çalışır:
```python
'run fast play footbal'
```
Sonuç olarak, kelimeler birleştirilir ve tek bir metin haline gelir.

### Neden Yapılır?
- Bu işlem, kelimelerin köklerine indirgenmesi sonrasında tekrar metin formatında kullanılabilir hale gelmesini sağlar. Stemming işleminden sonra kelimeleri tekrar birleştirerek, doğal dil işleme (NLP) modelinin anlayabileceği bir format elde edilir.

In [66]:
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [67]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [68]:
print(Y)

[1 0 1 ... 0 1 1]


In [69]:
Y.shape

(20800,)

In [70]:
# TfidfVectorizer sınıfını kullanarak metin verisini sayısal verilere dönüştürmek için bir vektörleştirici (vectorizer) oluşturuyoruz
vectorizer = TfidfVectorizer()

# X veri setindeki tüm metinleri öğrenmesi için TfidfVectorizer'a fit ediyoruz
vectorizer.fit(X)

# X veri setindeki metinleri sayısal verilere dönüştürüyoruz (vektörlere dönüştürme işlemi)
X = vectorizer.transform(X)

  
#### **bu adımları yaptık çünkü**:
- Metin verisini sayısal formata dönüştürmek zorundayız.
- **TfidfVectorizer**, metin verisindeki önemli kelimeleri vurgular ve modelin anlaması için metni uygun bir sayısal formata dönüştürür.
- Bu dönüşüm, makine öğrenmesi algoritmalarının veriyi işleyebilmesini sağlar ve modelin doğru tahminlerde bulunmasını kolaylaştırır.

In [71]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [72]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

model = LogisticRegression()

model.fit(X_train, Y_train)

LogisticRegression()

In [73]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9865985576923076


In [74]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9790865384615385


In [75]:
# 'X_test' veri setinden 3. örneği seçiyoruz ve 'X_new' değişkenine atıyoruz
X_new = X_test[3]

# Modeli kullanarak 'X_new' verisi için tahmin yapıyoruz
prediction = model.predict(X_new)

# Tahmin edilen değeri ekrana yazdırıyoruz
print(prediction)

# Eğer tahmin sonucu 0 ise haberin gerçek olduğunu, 1 ise sahte olduğunu yazdırıyoruz
if (prediction[0] == 0):
    print('The news is Real')
else:
    print('The news is Fake')

[0]
The news is Real


In [76]:
print(Y_test[3])

0
